In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 10000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')

t_exact = data['t'].reshape(-1,1)

x_exact = data['x'].reshape(-1,1)

Exact_u = np.real(data['usol']).T

print('t_exact:',t_exact.shape)
print('x_exact:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

In [ ]:
X_exact, T_exact = np.meshgrid(x_exact,t_exact)
print('X_exact:',X_exact.shape)
print('T_exact:',T_exact.shape)

X_exact_flatten = X_exact.flatten()[:,None]
T_exact_flatten = T_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None] 

In [ ]:
data_star.shape

In [ ]:
X_star = numpy_to_tensor(data_star, var_name="X_star", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
X_star

In [ ]:
np.random.seed(5678)
n_test_data = 10000

index_t = (lhs(1,n_test_data))*len(t_exact)
index_t = np.floor(index_t).reshape(n_test_data,).astype(int)
observe_t = t_exact[index_t]

index_x = (lhs(1,n_test_data))*len(x_exact)
index_x = np.floor(index_x).reshape(n_test_data,).astype(int)
observe_x = x_exact[index_x]

test_data = np.hstack((observe_x,observe_t))
test_u = Exact_u[index_t,index_x].reshape(-1,1)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

In [ ]:
test_u

In [10]:
def output_transform(x, y):
    x_in = x[:, 0:1]
    t_in = x[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y - torch.sin(np.pi * x_in)

## Loss_f

In [11]:
def get_loss_f(x_grad, t_grad, PINNs, C, return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_grad,t_grad),1))
    #E_inside = output_transform(torch.cat((x_grad,y_grad),axis=1), E_inside)
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])  
    E_t = compute_higher_order_derivatives(E_inside, [t_grad])
    
    loss_term = E_t+E_inside*E_x-C1*E_xx
    ########### loss f  ###########
    
    if return_sequence == 'yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

## loss_bound

In [12]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):

    E_bound = PINNs1(torch.cat((bound_x,torch.zeros_like(bound_x)),1))
    real_bound = -torch.sin(torch.tensor(np.pi)*bound_x)
    
    loss_bound_for_a = torch.mean(torch.square(E_bound-real_bound))
    
    loss_bound_for_b = torch.mean(torch.square(PINNs1(torch.cat((torch.ones_like(bound_t),bound_t),1))))
    
    loss_bound_for_c = torch.mean(torch.square(PINNs1(torch.cat((-torch.ones_like(bound_t),bound_t),1))))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA on GPU")
else:
    device = "cpu"
    print("Using CPU")

# PINN

In [14]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [15]:
net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[20]*8, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [16]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_b_1 = []
test_loss_1 = []
############## Record list ###############

In [ ]:
C1 = 0.01/np.pi

nIter1 = 20000
it = 0

while it<20000:
    
    ##### loss bound  ######
    loss_bound = get_loss_bound(x_bound, t_bound, PINNs1, C1, return_sequence='not')

    ############## loss f ###############
    loss_f = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    ############## loss f ###############    
    
    #########loss PI#########
    loss = loss_f+loss_bound
   
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)  
    test_loss = relative_l2_torch(pre_u,test_u)
    #########  test_loss NRMSE  #########
    
    ##############Record###############
    loss_all_1.append(loss.item())
    loss_f_1.append(loss_f.item())
    loss_b_1.append(loss_bound.item()) 
    test_loss_1.append(test_loss)
    ##############Record Original###############
    
    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss) 

    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1   

print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)